In [1]:
print("\n\nம ஹா ம்ரி த் யு ஞ் ஜ யா\n\n")



ம ஹா ம்ரி த் யு ஞ் ஜ யா




In [2]:
from scipy import stats

In [3]:
#import tensorflow as tf

import pickle
import pandas as pd
import os
from matplotlib import pyplot
from pathlib import Path
#from tensorflow.contrib.rnn import LSTMCell, GRUCell
import numpy as np
import time, uuid
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
#

In [4]:
def Box_filter(values, kernel_size=3):
    """
    :return: The list of filtered average
    """
    filter_values = np.cumsum(values, dtype=float)

    filter_values[kernel_size:] = filter_values[kernel_size:] - filter_values[:-kernel_size]
    filter_values[kernel_size:] = filter_values[kernel_size:] / kernel_size

    for i in range(1, kernel_size):
        filter_values[i] /= i + 1

    return filter_values

In [5]:
def extrapolate_next(values):
    """
    Extrapolates the next value by sum up the slope of the last value with previous values.
    :param values: a list or numpy array of time-series
    :return: the next value of time-series
    """

    last_value = values[-1]
    slope = [(last_value - v) / i for (i, v) in enumerate(values[::-1])]
    slope[0] = 0
    next_values = last_value + np.cumsum(slope)

    return next_values

In [6]:
def merge_series(values, extend_num=5, forward=5):

    next_value = extrapolate_next(values)[forward]
    extension = [next_value] * extend_num

    if isinstance(values, list):
        merge_values = values + extension
    else:
        merge_values = np.append(values, extension)
    return merge_values

In [7]:
class Silency(object):
    def __init__(self, amp_window_size, series_window_size, score_window_size):
        self.amp_window_size = amp_window_size
        self.series_window_size = series_window_size
        self.score_window_size = score_window_size

    def frequency_domain_map(self, values):
        freq = np.fft.fft(values)
        mag = np.sqrt(freq.real ** 2 + freq.imag ** 2)
        spectral_residual = np.exp(np.log(mag) - Box_filter(np.log(mag), self.amp_window_size))

        freq.real = freq.real * spectral_residual / mag
        freq.imag = freq.imag * spectral_residual / mag

        freqmap = np.fft.ifft(freq)
        freq_map = np.sqrt(freqmap.real ** 2 + freqmap.imag ** 2)
        return freq_map

    def generate_anomaly_score(self, values, type="avg"):
        extended_series = merge_series(values, self.series_window_size, self.series_window_size)
        mag = self.frequency_domain_map(extended_series)[: len(values)]

        ave_filter = Box_filter(mag, self.score_window_size)        
        score = (mag - ave_filter) / ave_filter 
        
        return score

    def generate_anomaly_score2(self, values, type="avg"):
        extended_series = merge_series(values, self.series_window_size, self.series_window_size)
        mag = self.frequency_domain_map(extended_series)[: len(values)]

        return mag

    def generate_anomaly_score_(self, values, type="avg"):
        extended_series = merge_series(values, self.series_window_size, self.series_window_size)
        mag = self.frequency_domain_map(extended_series)[: len(values)]

        ave_filter = Box_filter(mag, self.score_window_size)
        
        ave_filter1 = Box_filter(ave_filter, 12)
        ave_filter2 = Box_filter(ave_filter1, 8)
        ave_filter = Box_filter(ave_filter2, 6)
        
        score = (mag - ave_filter) / ave_filter
        
        return score
    

# Entire A4 folder

In [8]:
path = '/Users/manjunath.adinarayan/TIME-SERIES/YAHOO_Data/Full_data/'


In [9]:
import os
import glob

extension='csv'
os.chdir( path )
files = glob.glob('*.{}'.format(extension))
len(files)

367

In [10]:
files

['real_59.csv',
 'real_65.csv',
 'A3Benchmark-TS12.csv',
 'synthetic_85.csv',
 'synthetic_91.csv',
 'A4Benchmark-TS99.csv',
 'A4Benchmark-TS72.csv',
 'synthetic_46.csv',
 'synthetic_52.csv',
 'A4Benchmark-TS66.csv',
 'synthetic_53.csv',
 'A4Benchmark-TS67.csv',
 'A4Benchmark-TS73.csv',
 'synthetic_47.csv',
 'A4Benchmark-TS98.csv',
 'synthetic_90.csv',
 'synthetic_84.csv',
 'A3Benchmark-TS13.csv',
 'real_64.csv',
 'real_58.csv',
 'real_66.csv',
 'A3Benchmark-TS11.csv',
 'A3Benchmark-TS39.csv',
 'synthetic_92.csv',
 'synthetic_86.csv',
 'A4Benchmark-TS65.csv',
 'synthetic_51.csv',
 'synthetic_45.csv',
 'A4Benchmark-TS71.csv',
 'A4Benchmark-TS59.csv',
 'synthetic_79.csv',
 'synthetic_78.csv',
 'synthetic_100.csv',
 'A4Benchmark-TS58.csv',
 'synthetic_44.csv',
 'A4Benchmark-TS70.csv',
 'A4Benchmark-TS64.csv',
 'synthetic_50.csv',
 'synthetic_87.csv',
 'synthetic_93.csv',
 'A3Benchmark-TS38.csv',
 'A3Benchmark-TS10.csv',
 'real_67.csv',
 'real_63.csv',
 'A3Benchmark-TS28.csv',
 'A3Benchmark

In [11]:
# less than period
amp_window_size=4
# (maybe) as same as period
series_window_size=4
# a number enough larger than period
score_window_size=16

spec = Silency(amp_window_size, series_window_size, score_window_size)

In [12]:

prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df = pd.read_csv(os.path.join(path,file), header=0)

    test_signal = df.value.values 

    score = spec.generate_anomaly_score_(test_signal)

    index_changes = np.where(score > np.percentile(score, 99))[0]

    cutoff = np.percentile(score, 99)

    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
#    pred_anom = np.zeros(len(test_signal))
#    for i in range(int(len(test_signal))):
#        for j in index_changes:
#            pred_anom[j] = 1

    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )


/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


In [13]:
np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

(0.40723272960410317, 0.8852972149525491, 0.5047092172414838)

In [14]:
import time

# EXPERIMENTS: 

In [15]:
# less than period
amp_window_size=3
# (maybe) as same as period
series_window_size=3
# a number enough larger than period
score_window_size=32

spec = Silency(amp_window_size, series_window_size, score_window_size)

# 1

# fixed threshold '4'

# BEST parameter set, F1_score = 74.57%

In [16]:
path='/Users/manjunath.adinarayan/TIME-SERIES/YAHOO_Data/A4Benchmark/'
path = '/Users/manjunath.adinarayan/TIME-SERIES/YAHOO_Data/Full_data/'

extension='csv'
os.chdir( path )
files = glob.glob('*.{}'.format(extension))
len(files)

367

### FULL DATA

In [17]:
st = time.time()
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df = pd.read_csv(os.path.join(path,file), header=0)
    test_signal = df.value.values 
    score = spec.generate_anomaly_score(test_signal)
    index_changes = np.where(score > np.percentile(score, 99))[0]
    cutoff = np.percentile(score, 99)
    cutoff = 4
    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

print("Time taken =", (time.time()-st), "seconds")
np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list), len(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Time taken = 3.6278698444366455 seconds


(0.7787483979520454, 0.8016182114371371, 0.7457641660484401, 367)

In [18]:
# Individual Fscore of Yahoo's A1, A2, A3, A4
#A1 = 32.23%
#A2 = 73.008%
#A3 = 91.19%
#A4 = 87.89%

### HALF DATA

In [38]:
path='/Users/manjunath.adinarayan/TIME-SERIES/YAHOO_Data/A4Benchmark/'
path = '/Users/manjunath.adinarayan/TIME-SERIES/YAHOO_Data/Full_data/'

extension='csv'
os.chdir( path )
files = glob.glob('*.{}'.format(extension))
len(files)

367

In [39]:
st = time.time()
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df1 = pd.read_csv(os.path.join(path,file), header=0)
    df = df1.iloc[int(len(df1)/2):, :]
    test_signal = df.value.values 
    score = spec.generate_anomaly_score(test_signal)
    index_changes = np.where(score > np.percentile(score, 99))[0]
    cutoff = np.percentile(score, 99)
    cutoff = 4
    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

print("Time taken =", (time.time()-st), "seconds")
        
np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


Time taken = 3.999303102493286 seconds


(0.784573328578778, 0.7552290545053432, 0.7320365590665248)

In [ ]:
# Individual Fscore of Yahoo's A1, A2, A3, A4
#A1 = 27.78%
#A2 = 78.6%
#A3 = 89.25%
#A4 = 82.14%

# 2

# Enhanced SR with no averaging/filtering of saliency map
# and
# cutoff = 7 * np.mean(score)

def generate_anomaly_score(self, values, type="avg"):

        extended_series = merge_series(values, self.series_window_size, self.series_window_size)
        
        mag = self.frequency_domain_map(extended_series)[: len(values)]
        
        return mag

In [20]:
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df = pd.read_csv(os.path.join(path,file), header=0)
    test_signal = df.value.values 
    score = spec.generate_anomaly_score2(test_signal)
    index_changes = np.where(score > np.percentile(score, 99))[0]
    cutoff = np.percentile(score, 99)
    cutoff = 7 * np.mean(score)
    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


(0.6509540021083907, 0.7779652423245614, 0.6508797391830556)

In [21]:
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df1 = pd.read_csv(os.path.join(path,file), header=0)
    df = df1.iloc[int(len(df1)/2):, :]
    test_signal = df.value.values 
    score = spec.generate_anomaly_score2(test_signal)
    index_changes = np.where(score > np.percentile(score, 99))[0]
    cutoff = np.percentile(score, 99)
    cutoff = 7 * np.mean(score)
    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


(0.675088970218398, 0.7560466291248261, 0.6570270138255867)

# 3

# Enhanced SR with averaging/filtering of saliency map
# cutoff = 2.8

In [22]:
#def generate_anomaly_score(self, values, type="avg"):
#    extended_series = merge_series(values, self.series_window_size, self.series_window_size)
#    mag = self.frequency_domain_map(extended_series)[: len(values)]
#    ave_filter = Box_filter(mag, self.score_window_size)
#    score = (mag - ave_filter) / ave_filter
#    return score

In [23]:
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df = pd.read_csv(os.path.join(path,file), header=0)
    test_signal = df.value.values 
    score = spec.generate_anomaly_score(test_signal)
    #index_changes = np.where(score > np.percentile(score, 99))[0]
    cutoff = 2.8
    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


(0.5968055594659185, 0.8690790440124405, 0.6590783629749551)

In [24]:
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df1 = pd.read_csv(os.path.join(path,file), header=0)
    df = df1.iloc[int(len(df1)/2):, :]
    test_signal = df.value.values 
    score = spec.generate_anomaly_score(test_signal)
    #index_changes = np.where(score > np.percentile(score, 99))[0]
    cutoff = 2.8
    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


(0.6589798720628981, 0.8312036686714798, 0.6940316418644611)

# Half of the data [ TEST ]

In [25]:
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df1 = pd.read_csv(os.path.join(path,file), header=0)
    df = df1.iloc[int(len(df1)/2): , : ]
    test_signal = df.value.values 
    score = spec.generate_anomaly_score(test_signal)
    index_changes = np.where(score > np.percentile(score, 99))[0]
    cutoff = np.percentile(score, 99)
    cutoff = 4
    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)
            
    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


(0.784573328578778, 0.7552290545053432, 0.7320365590665248)

In [26]:
prec_list, rec_list, f1score_list = [], [], []

for file in files:
    df = pd.read_csv(os.path.join(path,file), header=0)

    test_signal = df.value.values

    score = spec.generate_anomaly_score(test_signal)

    index_changes = np.where(score > np.percentile(score, 98))[0]

    cutoff = np.percentile(score, 98)

    pred_anom = []
    for i in range(0, len(score)):
        if score[i] > cutoff:
            pred_anom.append(1)
        else:
            pred_anom.append(0)

    if "Benchmark" in file:
        prec_list.append(precision_score(df.anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.anomaly.values, pred_anom) )
    else:
        prec_list.append(precision_score(df.is_anomaly.values, pred_anom) )
        rec_list.append(recall_score(df.is_anomaly.values, pred_anom) )
        f1score_list.append(f1_score(df.is_anomaly.values, pred_anom) )

np.mean(prec_list), np.mean(rec_list), np.mean(f1score_list)

/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in double_scalars
  if __name__ == '__main__':
/Users/manjunath.adinarayan/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in long_scalars
  if __name__ == '__main__':


(0.20945774908666842, 0.8896150391957784, 0.3147837013819169)

In [27]:
len(files)

367

In [28]:
cnt=0
for file in files:
    if "Benchmark" in file:
        cnt = cnt + 1
print(cnt)

200
